In [5]:
import spacy
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
      --------------------------------------- 0.3/12.8 MB ? eta -:--:--
     - ------------------------------------- 0.5/12.8 MB 284.3 kB/s eta 0:00:44
     - ------------------------------------- 0.5/12.8 MB 284.3 kB/s eta 0:00:44
     - ------------------------------------- 0.5/12.8 MB 284.3 kB/s eta 0:00:44
     - ------------------------------------- 0.5/12.8 MB 284.3 kB/s eta 0:00:44
     -- ------------------------------------ 0.8/12.8 MB 299.6 kB/s eta 0:00:41
     -- ----------------

In [7]:
import spacy
from spacy.matcher import PhraseMatcher

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# ✅ Predefined list of symptoms & injuries (expandable)
SYMPTOM_KEYWORDS = [
    "bleeding", "fracture", "wound", "limping", "shivering", 
    "vomiting", "injury", "burn", "pus", "weakness", "cough", "blood"
]

# Convert keywords to spaCy's PhraseMatcher
matcher = PhraseMatcher(nlp.vocab)
patterns = [nlp(text) for text in SYMPTOM_KEYWORDS]
matcher.add("SYMPTOMS", patterns)

def extract_entities(text):
    doc = nlp(text)
    symptoms = []
    location = None

    # Match symptoms in text
    matches = matcher(doc)
    for match_id, start, end in matches:
        symptoms.append(doc[start:end].text)

    # Extract possible locations
    for ent in doc.ents:
        if ent.label_ in ["GPE", "LOC"]:
            location = ent.text

    return {"symptoms": list(set(symptoms)), "location": location}

# Example user input
user_input = "blood, shivering, and lying on the side of the road"
entities = extract_entities(user_input)
print(entities)


{'symptoms': ['blood', 'shivering'], 'location': None}


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC

# Sample training data (expand this for better accuracy)
X_train = [
    "profuse bleeding, unconscious",  # High urgency
    "broken leg, limping",  # Medium urgency
    "shivering, minor wound",  # Medium urgency
    "healthy, walking normally",  # Low urgency
]
y_train = ["HIGH", "MEDIUM", "MEDIUM", "LOW"]

# Convert text to feature vectors
vectorizer = CountVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)

# Train SVM classifier
clf = SVC(kernel="linear")
clf.fit(X_train_vectors, y_train)

# Predict urgency level for new input
X_test = [user_input]
X_test_vector = vectorizer.transform(X_test)
predicted_urgency = clf.predict(X_test_vector)[0]

print("Predicted Urgency:", predicted_urgency)


Predicted Urgency: MEDIUM


In [9]:
def generate_whatsapp_report(data, entities, urgency):
    case_id = f"SH-20250308-{str(data['case_id']).zfill(5)}"
    image_url = f"[View Reported Case]({data['image_path']})"
    
    symptoms_text = ", ".join(entities["symptoms"]) if entities["symptoms"] else "No visible injury"
    location_text = entities["location"] if entities["location"] else "(Location not provided)"

    report = f"""
📢 *Urgent Stray Animal Assistance Required!*

🆔 *Case ID:* {case_id}
📍 *Location:* {location_text}
🖼 *Image:* {image_url}
🔍 *Detection:* {data['prediction']} ({symptoms_text})
⚠ *Urgency Level:* {urgency} (Review needed)

🚀 *Action Required:* Please review the case and initiate rescue intervention.
📞 *Contact Person (If Available):* Sanket Suryawanshi (+91XXXXXXXXXX)

🔗 [Update Case Status Here](NGO Dashboard)

This message was auto-generated by *Stray Help* – A Technology-Driven Platform for Stray Animal Welfare.
    """
    
    return report.strip()

# Sample Data
data = {
    "case_id": 10,
    "image_path": "uploads/85120553.webp",
    "prediction": "Not Injured",
    "user_input": "blood, shivering, and lying on the side of the road"
}

# Generate final report
final_report = generate_whatsapp_report(data, entities, predicted_urgency)
print(final_report)


📢 *Urgent Stray Animal Assistance Required!*

🆔 *Case ID:* SH-20250308-00010
📍 *Location:* (Location not provided)
🖼 *Image:* [View Reported Case](uploads/85120553.webp)
🔍 *Detection:* Not Injured (blood, shivering)
⚠ *Urgency Level:* MEDIUM (Review needed)

🚀 *Action Required:* Please review the case and initiate rescue intervention.
📞 *Contact Person (If Available):* Sanket Suryawanshi (+91XXXXXXXXXX)

🔗 [Update Case Status Here](NGO Dashboard)

This message was auto-generated by *Stray Help* – A Technology-Driven Platform for Stray Animal Welfare.
